In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

In [2]:
import matplotlib
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 15}

matplotlib.rc('font', **font)

## Process labelling results 

In [3]:
ordered_verts = ['T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'L1', 'L2', 'L3', 'L4']

In [18]:
root_dir = '/data/PAB_data/vert_labelling/'
filename = 'labeller_preds.npz'

pred_dict = {}
gt_dict = {}
fold_dict = {}
img_dict = {}
heatmap_dict = {}
coords_dict = {}
labels_dict = {}
for root, dirs, files in os.walk(root_dir):
    if filename in files:
        fold = root.split('/')[-1]
        print(fold)
        pred = np.load(os.path.join(root, filename))
        # Collect targets
        tgt_path = os.path.join(root, 'targets/coordinates/')
        for file in os.listdir(tgt_path):
            name = file.split('.')[0]
            gt_dict[name] = pd.read_csv(os.path.join(tgt_path, file))
        # Collect input image
        img_path = os.path.join(root, 'slices')
        for file in os.listdir(img_path):
            name = file.split('.')[0]
            img_dict[name] = np.load(os.path.join(img_path, file))
        # Collect predictions
        for id_, mask, coords, heatmaps in zip(pred['ids'], pred['masks'], 
                                                    pred['coords'], pred['heatmaps']):
            pred_dict[id_] = mask
            fold_dict[id_] = fold
            coords_dict[id_] = coords
            heatmap_dict[id_] = heatmaps
print(len(pred_dict.keys()), len(list(gt_dict.keys())))

q1
100 100


In [19]:
ids = list(pred_dict.keys())

## Check preds

In [44]:
@interact
def plot_preds(name=ids):
    idx = ids.index(name)
    fig, ax = plt.subplots(1, 2, figsize=(20, 20))
    ax[0].axis('off')
    ax[1].axis('off')
    coords = coords_dict[name]
    gt_coords = gt_dict[name]
    ax[0].imshow(img_dict[name][..., 1], cmap='gray')
    ax[1].imshow(img_dict[name][..., 1], cmap='gray')
    for i, vert in enumerate(ordered_verts):
        y, x = coords[i]
        ax[1].scatter(x, y, marker='+', c='y', s=15)
        ax[1].text(x, y, vert, c='y', size=15)
    for i, vert in enumerate(ordered_verts):
        if vert not in pd.unique(gt_coords['Level']): continue
        tmp =  gt_coords.loc[gt_coords['Level'] == vert].values[0]
        vert, x, y = tmp
        ax[0].scatter(x, y, marker='+', c='y', s=15)
        ax[0].text(x, y, vert, c='y', size=15)
    plt.subplots_adjust(wspace=0)

interactive(children=(Dropdown(description='name', options=('fr_511_LS_Sag', '03_06_2014_399_LS_Sag', 'fr_534_…